# Import Library Dependencies

In [1]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
# MACD/EMA from TA
from ta.trend import MACD, EMAIndicator
# Boll from TA
from ta.volatility import BollingerBands

# Import S&P500 Stock Tickers

In [53]:
# Read csv that contains the tickers
# https://stockmarketmba.com/stocksinthesp500.php
stocks = pd.read_csv('Stocks in the SP 500 Index.csv')
stocks.set_index('Symbol',inplace=True)
stocks.head()

,Description,Category2,Category3,GICS Sector,Market cap,Dividend yield,Price to TTM earnings,Price to TTM sales,Price to book value,Action
Symbol,,,,,,,,,,
AAPL,Apple Inc,Common stocks,Large cap,Information Technology,"$2,783,293,538,880",0.52%,0.0,0.0,0.0,Analyze
MSFT,Microsoft Corp,Common stocks,Large cap,Information Technology,"$2,188,667,527,676",0.82%,0.0,0.0,0.0,Analyze
AMZN,Amazon.Com Inc.,Common stocks,Large cap,Consumer Discretionary,"$1,458,654,098,841",0.00%,0.0,0.0,0.0,Analyze
TSLA,Tesla Inc,Common stocks,Large cap,Consumer Discretionary,"$969,244,954,319",0.00%,0.0,0.0,0.0,Analyze
BRK.B,Berkshire Hathaway Inc. Class B,Common stocks,Large cap,Financials,"$667,074,494,443",0.00%,0.0,0.0,0.0,Analyze


# Get Stock Data from Yahoo Finance

In [54]:
yf_response = yf.download(stocks.index.tolist(),period='1d')['Close']
yf_response

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-08-18 00:00:00,139.970001,14.84,207.490005,174.149994,141.289993,149.910004,278.179993,109.959999,319.459991,439.029999,...,65.510002,76.989998,94.379997,36.360001,102.349998,117.949997,114.519997,342.380005,58.790001,170.169998


In [60]:
# Delete the stocks failed to be downloaded (might have been delisted: BF.B, BRK.B)
stocks = stocks.drop(index=['BF.B','BRK.B'])

# Add Data to Dataframe

In [61]:
# Create new dataframe to add data
header=['Ticker','Price','MC','Shares2Buy']
df = pd.DataFrame(columns=header)

for stock in stocks.index.tolist():
        df.loc[len(df.index)] =  pd.Series([
            stock,
            yf_response.loc[:,stock].iloc[0],
            stocks.loc[stock,'Market cap'],
            'N/A'
        ],
            index=header,
        )
    
df.head()

,Ticker,Price,MC,Shares2Buy
0,AAPL,174.149994,"$2,783,293,538,880",N/A
1,MSFT,290.170013,"$2,188,667,527,676",N/A
2,AMZN,142.300003,"$1,458,654,098,841",N/A
3,TSLA,908.609985,"$969,244,954,319",N/A
4,META,174.660004,"$516,260,083,154",N/A


# Calculate Your Portfolio equaly weighted with S&P500

In [69]:
while True:
    try:
        portfolio_size = float(input('Enter the size of your portfolio in $:'))
        break
    except:
        print('Only numerical value accepted.')

Enter the size of your portfolio in $: 1000000


In [80]:
# position_size spent on each stock
position_size = portfolio_size/len(df.index)
for i in range(len(df.index)):
    df.loc[i, 'Shares2Buy'] = position_size // df.loc[i, 'Price']
df

,Ticker,Price,MC,Shares2Buy
0,AAPL,174.149994,"$2,783,293,538,880",11.0
1,MSFT,290.170013,"$2,188,667,527,676",6.0
2,AMZN,142.300003,"$1,458,654,098,841",14.0
3,TSLA,908.609985,"$969,244,954,319",2.0
4,META,174.660004,"$516,260,083,154",11.0
...,...,...,...,...
496,ALK,46.689999,"$6,109,148,873",42.0
497,PENN,35.720001,"$5,938,166,936",55.0
498,VNO,29.52,"$5,757,087,271",67.0
499,PVH,70.199997,"$4,586,127,316",28.0
